In [1]:
%reload_ext autoreload
%autoreload 2

import partialkey
import numpy as np
from random import randrange
from Crypto.Cipher import AES
import aescore
import base64
import math
import cluster
import time

np.set_printoptions(formatter={'int':hex})

Searchkey = partialkey.searchkey()

PCI is being Initialized....
PCI Initialized
BAR0 Size: 0x10000
BAR1 Size: 0x100000
pci_status : 0xffffffffe77232feL
pci_command : 0xffffffff000000feL
pci_status : 0xe7723200L
pci_command : 0x100100L
setting node 0 with address : 0
setting node 1 with address : 4
setting node 2 with address : 8


In [2]:
key = np.array([0xFFFFFFFFFFFFFF00, 0x08090a0b0c0d0e0f,
                0x1011121314151617, 0x18191a1b1c1d1e1f,], dtype=np.uint64)
data = "Abhishek  Bajpai"
mode = AES.MODE_ECB
aes = AES.new(key, mode)
aes.use_aesni=True
crypto = base64.b64encode(aes.encrypt(data))
text = aes.decrypt(base64.b64decode(crypto))
#print text
## partial key delete
searchptr = 0
searchsize = 34
mask0, mask1, keyindex = partialkey.createmask(searchptr, searchsize)
key[keyindex] = int(key[keyindex]) & (0xFFFFFFFFFFFFFFFF ^ mask0)
key[keyindex+1] = int(key[keyindex+1]) & (0xFFFFFFFFFFFFFFFF ^ mask1)
text = str.encode(str(data))
#print key
c = np.frombuffer(base64.b64decode(crypto), dtype=np.uint64)

text = aes.decrypt(c)
#print c

In [ ]:
import datetime
t1 = datetime.datetime.now()
res, reskey = Searchkey.search(key, text, crypto, searchptr = searchptr, searchsize = searchsize)
if res == 1:
    print reskey
    print "search done"
t2 = datetime.datetime.now()
t3 = t2-t1
print "\nExecution time: %s seconds" % (t3.total_seconds())

>> 13
>> 12
>> 9
>> 8
>> 5
>> 4
>> 2
>> 1
{1: {'grp': 248, 'ctr': 0}, 2: {'grp': 249, 'ctr': 0}, 4: {'grp': 250, 'ctr': 0}, 5: {'grp': 251, 'ctr': 0}, 8: {'grp': 252, 'ctr': 0}, 9: {'grp': 253, 'ctr': 0}, 12: {'grp': 254, 'ctr': 0}, 13: {'grp': 255, 'ctr': 0}}
[]
$$$$$$$$$$$$$$ 13
{'searchptr': 8, 'key': array([0xfffffffc000000ffL, 0x8090a0b0c0d0e0fL, 0x1011121314151617L,
       0x18191a1b1c1d1e1fL], dtype=uint64), 'packet': array([0x60f0000080381aeL, 0xfffffffc000000ffL, 0x8090a0b0c0d0e0fL,
       0x1011121314151617L, 0x18191a1b1c1d1e1fL, 0x6b65687369686241L,
       0x6961706a61422020L, 0x5ea7fb51f36d24a5L, 0x20a96677fa9b6b2bL], dtype=uint64), 'packetid': 1551}
$$$$$$$$$$$$$$ 12
{'searchptr': 8, 'key': array([0xfffffffc000000feL, 0x8090a0b0c0d0e0fL, 0x1011121314151617L,
       0x18191a1b1c1d1e1fL], dtype=uint64), 'packet': array([0x3c100000080381aeL, 0xfffffffc000000feL, 0x8090a0b0c0d0e0fL,
       0x1011121314151617L, 0x18191a1b1c1d1e1fL, 0x6b65687369686241L,
       0x6961706a61422020

In [ ]:
import datetime
t1 = datetime.datetime.now()
Searchkey.search_thread(key, text, crypto, searchptr = searchptr, searchsize = searchsize)

for i in range(10):
    print Searchkey.get_status()
    
Searchkey.search_thread_stop()

In [3]:
import time
from multiprocessing import Process, Queue

def f(q):
    for i in range(5):
        q.put([42, None, 'hello'])
        time.sleep(1)
    print 'returning'
    return 0

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    for i in range(5):
        print q.get()
        time.sleep(1)     
    p.join()

[42, None, 'hello']
[42, None, 'hello']
[42, None, 'hello']
[42, None, 'hello']
[42, None, 'hello']
returning


In [ ]:
p.join()